In [1]:
import os

os.chdir("../..")

In [2]:
from src.dataset import classification

classification_df = classification.get_oneshot_classification_df()
classification_df.shape

(17777, 1)

In [3]:
classification_df.head()

,image_path
id,
%ce%b4-rainbow-energy-149-pcg-p-japanese-promo,db/shared/jpg/pokumon/%ce%b4-rainbow-energy-14...
%ce%b4-rainbow-energy-9-17-pokemon-organized-play-events-pop-series-5,db/shared/jpg/pokumon/%ce%b4-rainbow-energy-9-...
1st-place-rare-candy-welcome-festa-2017-sm-p,db/shared/jpg/pokumon/1st-place-rare-candy-wel...
20-damage-from-confusion-expansion-sheet-3-1998-vending,db/shared/jpg/pokumon/20-damage-from-confusion...
2nd-place-rare-candy-welcome-festa-2017-sm-p,db/shared/jpg/pokumon/2nd-place-rare-candy-wel...


In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from PIL import Image
import pandas as pd
import os

In [5]:
import clip

class PokemonCardsDataset(Dataset):
    def __init__(self, df):
        self.df = df

        self.clip_model, self.preprocess_clip = clip.load("ViT-B/32", device="cpu")
        self.clip_model.eval()
        self.device = "cuda" if torch.cuda.is_available() else "cpu"

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_path = self.df.iloc[idx]['image_path']
        img = Image.open(img_path).convert('RGB')

        img_tensor = transforms.ToTensor()(img)

        with torch.no_grad():
            img_embedding = self.preprocess_clip(img_tensor.unsqueeze(0)).squeeze()

        return img_tensor, img_embedding